In [1]:
from MIMIC3 import MIMIC_3
from ICDCodesGrouper import ICDCodesGrouper
import pandas as pd

In [2]:
mimic_path =  "/notebooks/Experiments/Datasets/MIMIC 3/"
icdgrouper = ICDCodesGrouper(ccs_path='CCS-SingleDiagnosisGrouper.txt')

In [3]:
dataset = lambda x: f'{mimic_path}/{x}.csv.gz'

In [ ]:
pd.read_csv(dataset('ADMISSIONS'))

In [7]:
mimic = MIMIC_3(mimic_path,save_steps = True)
grouper = lambda group: lambda x: icdgrouper.lookup(group,x)
grouper = grouper('ccs')
df = mimic.get_tabular()

In [8]:
mimic.get_label(base=mimic.to_self_supervised())

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [7]:
b.sum()

5854

In [9]:
a.merge(b,right_index=True,left_index=True,how='inner')['DEATHTIME'].sum()

1445

In [102]:
df.ICD9_CODE = df.ICD9_CODE.apply(grouper)

In [103]:
df.ICD9_CODE.nunique()

281

In [104]:
freq = df.groupby('ICD9_CODE')["HADM_ID"].apply(len)
n_visits = df.HADM_ID.nunique()

In [105]:
pct_frequence = (freq/freq.max())*100

In [106]:
missing = icdgrouper.get_classes('ccs')[[item not in pct_frequence.index for item in icdgrouper.get_classes('ccs')]]
missing

array([194, 254])

In [107]:
pct_frequence = pd.concat([pct_frequence,pd.Series(index=missing).fillna(0)]).sort_index()


/tmp/ipykernel_4002/2388863096.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pct_frequence = pd.concat([pct_frequence,pd.Series(index=missing).fillna(0)]).sort_index()


In [108]:
pct_frequence

1        0.584645
2       49.055008
3       23.356376
4        7.707016
5        3.202677
          ...    
2617    18.638770
2618     1.581115
2619     0.466980
2620     1.706133
2621     8.023239
Length: 283, dtype: float64

In [109]:
bins = pd.cut(pct_frequence,bins=5)

In [110]:
bins

1       (-0.1, 20.0]
2       (40.0, 60.0]
3       (20.0, 40.0]
4       (-0.1, 20.0]
5       (-0.1, 20.0]
            ...     
2617    (-0.1, 20.0]
2618    (-0.1, 20.0]
2619    (-0.1, 20.0]
2620    (-0.1, 20.0]
2621    (-0.1, 20.0]
Length: 283, dtype: category
Categories (5, interval[float64, right]): [(-0.1, 20.0] < (20.0, 40.0] < (40.0, 60.0] < (60.0, 80.0] < (80.0, 100.0]]

In [12]:
from models import OutcomeModel
import torch
from utilities import *
from Processing import get_or_create_diagnosis_ccs_graph,init_embeddings
import networkx as nx

In [30]:
def load_model(path="./models/mimic/normal/ccs/50/256",embedding_size=128,n_layers=1,device='cpu'):
        base = lambda e,l :f'{path}/SW-Phenotype-{l}-{e}'
        model = OutcomeModel(embedding_size=embedding_size,n_layers=n_layers)
        model.load_state_dict(torch.load(base(embedding_size,n_layers),map_location=device))
        return model.to(device)

In [31]:
device = 'cuda'
embedding_size = 100
n_layers = 0

In [32]:
model = load_model(n_layers=0,embedding_size=100,device=device)


In [16]:
def setup_structure(path="./",embedding_size=256):
    G = nx.DiGraph()
    f_name = "./CCS-MultiDiagnosisGrouper.txt"
    G = get_or_create_diagnosis_ccs_graph(G,f_name,suffix=True)
    embeddings = init_embeddings(embedding_size,G=G) 
    return G,embeddings


In [17]:
G,embeddings = setup_structure(embedding_size=embedding_size)

/opt/conda/envs/pyg/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [18]:
mimic_path =  "/notebooks/Experiments/Datasets/MIMIC 3/"
icdgrouper = ICDCodesGrouper(ccs_path='CCS-SingleDiagnosisGrouper.txt')
mimic = MIMIC_3(mimic_path,save_steps = True)
grouper = lambda group: lambda x: icdgrouper.lookup(group,x)

target = 'ccs'

grouper = grouper(target)
df = mimic.to_self_supervised(target_grouper = grouper)

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

#TODO: not ready for replicate targets.
multi_hot = mlb.fit_transform(df.target)

existing_classes = mlb.classes_
all_classes = icdgrouper.get_classes(target)


not_common = np.array([idx for idx,i in enumerate(all_classes) if i not in existing_classes])
delta = all_classes.shape[0]-existing_classes.shape[0]


multi_hot = np.insert(multi_hot,not_common-(delta-1),0,axis=1) 

n_labels = multi_hot.shape[1]

df.target = list(multi_hot)

In [19]:
dataset = df.sample(10)
dataset

,diagnoses,target
SUBJECT_ID,,
1135,"[[41091, 4280, 5185, 5845, 9999, 486, 43820, 5...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
28644,"[[53110, 2639, 29181, 4589, 42789, 04186, 2886...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9688,"[[41401, 4111, 99812, 496, 60001, 25000, 5939,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
71774,"[[2252, 3485, 43491, 34202, 34551, 7843, 2724,...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
79539,"[[44024, 70715, 3310, 29410, V5867, 40390, V12...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8670,"[[41071, 4280, 25011, 41402, 25051, 36201, 250...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
19622,"[[4416, 7907, 5990, 70705, 99674, 5180, 70703,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
19164,"[[5672, 20410, 7907, 0414, 7994, 2830, 5722, 5...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6552,"[[1977, 452, 1976, 1961, 45620, 591, 2851, 576...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [20]:
x = dataset.diagnoses.values
y = np.array(list(dataset.target))


In [21]:
from graph_builders.diagnoses import DiagnosesGraph as Graph


In [22]:
validation = get_or_load_data(patients = x,target=y,G = G,embedding_size = embedding_size,graph_type=Graph,embeddings=embeddings,replicate=False,random=False).to(device)

In [29]:
validation.edge_index

tensor([[    0,     0,     0,  ..., 15934, 15935, 15936],
        [    0,     1,     2,  ..., 15928, 15928, 15928]], device='cuda:0')

In [70]:
model.eval()
with torch.no_grad():
    y_pred = model(validation)
y_real = validation.target

In [74]:
k =20
sort = y_pred.sort(axis=1,descending=True)
y_pred = sort.indices[:,0:k]
top_k = y_real.gather(1,y_pred)
div = torch.clamp(y_real.sum(dim=1),max=k)
    
recall = (top_k.sum(dim=1)/div).mean()
recall

tensor(0.0637, device='cuda:0')

In [111]:
y_pred *top_k
#iloc each index, sum 1
#I STILL DONT FULLY GRASP GRAM EVALUATION OMFG

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0., 18.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0., 17.,  0.,  0.,

In [90]:
bins

1       (-0.1, 20.0]
2       (40.0, 60.0]
3       (20.0, 40.0]
4       (-0.1, 20.0]
5       (-0.1, 20.0]
            ...     
2619    (-0.1, 20.0]
2620    (-0.1, 20.0]
2621    (-0.1, 20.0]
194     (-0.1, 20.0]
254     (-0.1, 20.0]
Length: 283, dtype: category
Categories (5, interval[float64, right]): [(-0.1, 20.0] < (20.0, 40.0] < (40.0, 60.0] < (60.0, 80.0] < (80.0, 100.0]]

In [84]:
top_k

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]], device='c